# Task
We want to achieve the following workflow:

Step 1. 
1. Write "Hello" in a file 

Step 2:
1. Check the file from step 1

**However, this is too simple for dflow. We want to write "Hello" to 10 different files in parallel.**

In [1]:
from typing import List

from dflow import Step, Workflow
from dflow.python import OP, OPIO, Artifact, OPIOSign, PythonOPTemplate, Slices

** Hello OP **
- input: 
    - 'filename': str that is the name of the file to generate

In [2]:
class Hello(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'filename': str
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign({
            'out_art': Artifact(str)
        })

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        file_num = int(op_in["filename"].split('.')[0][1:])
        with open(op_in["filename"], "w") as f:
            f.write("Hello" * file_num)
        op_out = OPIO({
            'out_art': op_in["filename"]
        })
        return op_out

**Check OP**
It is to check the files that have been written and print the files' pathes. `print(op_in["files"])` 

In [3]:
class Check(OP):
    def __init__(self):
        pass

    @classmethod
    def get_input_sign(cls):
        return OPIOSign({
            'files': Artifact(List[str])
        })

    @classmethod
    def get_output_sign(cls):
        return OPIOSign()

    @OP.exec_sign_check
    def execute(
            self,
            op_in: OPIO,
    ) -> OPIO:
        print(op_in["files"])
        return OPIO()

And then we can use OP above to write files repeatedly using Slices. This step is using Hello OP and python: 3.8 image.

**Slices**
We need to define input_parameter to be sliced and output_artifact to be stacked.

In [4]:
wf = Workflow("slices")
hello = Step("hello",
            PythonOPTemplate(Hello, image="python:3.8",
                            slices=Slices("{{item}}",
                                        input_parameter=["filename"],
                                        output_artifact=["out_art"]
                                        )
                            ),
            parameters={"filename": [f"f{x}.txt" for x in range(3)]},
            with_param=range(3))
check = Step("check",
            PythonOPTemplate(Check, image="python:3.8"),
            artifacts={"files": hello.outputs.artifacts["out_art"]},
            )
wf.add(hello)
wf.add(check)
wf.submit();

Workflow has been submitted (ID: slices-ldtjd, UID: 2842d9b9-9b1d-4319-aaff-d59866213278)
